### Loading Data

In [1]:
from keras.datasets import mnist
(x_train, labels_train), (x_test, labels_test) = mnist.load_data()

Using TensorFlow backend.


In [2]:
## The MNIST images are stored in the form of integers with values in the range [0,255]
## to convert to floating-point numbers in the range [0,1]
x_train = x_train.astype('float32')   # 
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [3]:
## The category labels are in the form of integers 0 to 9. 
## To define the output that the network should produce in response to each sample (a one hot encoding)
from keras.utils import to_categorical
y_train = to_categorical(labels_train, 10)
y_test = to_categorical(labels_test, 10)

In [4]:
## If the data is to be used as input to a dense layer, then it should be reshaped into a matrix where each row is a sample,
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [5]:
## If the data is to be used as input to a convolutional layer, 
## then it should be reshaped into a four dimensional matrix where the first dimension corresponds to the number of exemplars, 
## the second and third dimensions correspond to the width and height of each image, 
## and the fourth dimension corresponds to the number of colour channels in each image
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

### Defining a Neural Network

##### Method 1
- define a sequence of layers, it is assumed that the output of one layer provides the input to the next. For example, to build a three layer MLP network called “net”

In [6]:
from keras.models import Sequential
from keras.layers import Dense
net = Sequential()
net.add(Dense(800, activation='relu', input_shape=(784,)))  # 800 neurons (output units)
net.add(Dense(400, activation='relu'))
net.add(Dense(10, activation='softmax'))

Note that when adding a layer to the network, we can define parameters, such as: the number of neurons (in the example above the 1st layer has 800 neurons, the second has 400 neurons, and the 3rd layer has 10 neurons), and the activation function (in the example above the 1st and 2nd layers use RELU, and the 3rd layer uses softmax). Other activation functions are also available (see https://keras.io/activations/).

There are many different types of layers that can be used. (see: https://keras.io/layers/core/). For example, to build a simple CNN using convolutional, maxpooling, as well as dense layers (and with batch normalisation and dropout for some layers):

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization      
net = Sequential()
net.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
net.add(BatchNormalization())
net.add(Conv2D(64, (3, 3), activation='relu'))
net.add(MaxPool2D(pool_size=(2, 2)))
net.add(Flatten())
net.add(Dense(256, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(10, activation='softmax'))

### Training a Neural Network

In [9]:
net.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
history = net.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=256)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
14848/60000 [======>.......................] - ETA: 54s - loss: 0.7365 - acc: 0.7909

KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

In [9]:
history.history

{'acc': [0.9861000000317891, 0.9904333333015441, 0.9941999999682108],
 'loss': [0.04347277341882388, 0.02946271139383316, 0.018233727731804053],
 'val_acc': [0.9778, 0.9818, 0.9822],
 'val_loss': [0.07391852748282254, 0.06524046341339126, 0.06471494650582317]}

In [10]:
net.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 55us/step


[0.06471494482875569, 0.9822]

In [ ]:
## to show the change of cost function
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history.history['loss'], label='training loss')
# plt.plot(history.history['val_loss'], label='validation loss')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.legend()

### Testing a Neural Network

- 找出testing data的預測值(output)
- 跟true category labels做比較

In [ ]:
import numpy as np
outputs=net.predict(x_test)
labels_predicted=np.argmax(outputs, axis=1)
misclassified=sum(labels_predicted!=labels_test)
print('Percentage misclassified = ',100*misclassified/labels_test.size)